In [1]:
import requests
import pandas as pd

In [2]:
ds_cty = pd.DataFrame(pd.read_excel('DS_CTY.xlsx'))
array_info_DT_t_1 = ['3. Doanh thu thuần (1)-(2)']
array_info_TS_t_1 = ["TỔNG CỘNG TÀI SẢN"]
list_cty = ds_cty['MÃ CK'].to_list()
array_info_t_1 = [array_info_DT_t_1, array_info_TS_t_1]

In [3]:
url_DT = 'https://www.fireant.vn/api/Data/Finance/LastestFinancialReports?type=2&year=2019&quarter=0&count=6&symbol='
url_TS = 'https://svr5.fireant.vn/api/Data/Finance/LastestFinancialReports?type=1&year=2019&quarter=0&count=6&symbol='
url_list_t_1 = [url_DT, url_TS]
df_sale_t_1 = pd.DataFrame()
for index in range(0,len(list_cty)):
    df_financial = pd.DataFrame()
    for url_index in range(0, len(url_list_t_1)):
        url_temp = url_list_t_1[url_index] + list_cty[index]
        r = requests.get(url_temp)
        if r.status_code == 200:
            json_data = r.json()
        else:
            break
        if json_data != None:
            df = pd.DataFrame(json_data)
            df = df[df['Name'].isin(array_info_t_1[url_index])]
            if df.empty != True:
                df_financial = pd.concat([df_financial, df])
            else:
                df_financial = pd.DataFrame()
                break
        else:
            df_financial = pd.DataFrame()
            break

    if df_financial.empty != True :
        ten_cty_list = []
        for number in range(0, len(df_financial['Name'])):
            ten_cty_list.append(list_cty[index])
        df_financial['MÃ CK'] = ten_cty_list
        df_sale_t_1 = pd.concat([df_sale_t_1, df_financial])
        df_sale_t_1 = df_sale_t_1[['Name','MÃ CK','Values']]
df_sale_t_1.reset_index(drop=True,inplace=True)


In [4]:
df_sale_t_1.head()

,Name,MÃ CK,Values
0,3. Doanh thu thuần (1)-(2),AAA,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0..."
1,TỔNG CỘNG TÀI SẢN,AAA,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0..."
2,3. Doanh thu thuần (1)-(2),AAM,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0..."
3,TỔNG CỘNG TÀI SẢN,AAM,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0..."
4,3. Doanh thu thuần (1)-(2),AAT,"[{'Period': '2018', 'Year': 2018, 'Quarter': 0..."


In [5]:
for index, row in df_sale_t_1.iterrows():
    years = [2014,2015,2016,2017,2018,2019]
    if(index in df_sale_t_1.index):
        frame = df_sale_t_1.loc[index,'Values']
        year_in_frame = []
        for i in range(0,len(frame)) :
            year_in_frame.append(frame[i]['Year'])
        if(years == year_in_frame):
            for i in range(0,len(frame)):
                df_sale_t_1.loc[index,frame[i]['Year'] + 1] = frame[i]['Value']
        else:
            df_sale_t_1 = df_sale_t_1.drop(df_sale_t_1[df_sale_t_1['MÃ CK'] == df_sale_t_1.loc[index,'MÃ CK']].index)
df_sale_t_1.head()

,Name,MÃ CK,Values,2015,2016,2017,2018,2019,2020
0,3. Doanh thu thuần (1)-(2),AAA,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0...",1.560644e+12,1.614549e+12,2.143770e+12,4.069608e+12,8.011573e+12,9.258073e+12
1,TỔNG CỘNG TÀI SẢN,AAA,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0...",1.421651e+12,1.954765e+12,3.077616e+12,4.576157e+12,7.529167e+12,7.987454e+12
2,3. Doanh thu thuần (1)-(2),AAM,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0...",4.390167e+11,3.496310e+11,2.747782e+11,2.241892e+11,2.182639e+11,2.161219e+11
3,TỔNG CỘNG TÀI SẢN,AAM,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0...",2.971910e+11,3.219338e+11,2.579038e+11,2.515606e+11,2.299590e+11,2.275080e+11
8,3. Doanh thu thuần (1)-(2),ABT,"[{'Period': '2014', 'Year': 2014, 'Quarter': 0...",4.488608e+11,4.740068e+11,4.221049e+11,3.836953e+11,3.959384e+11,3.611504e+11


In [6]:
df_sale_t_1 = df_sale_t_1.drop(columns='Values')
df_sale_t_1 = df_sale_t_1.melt(id_vars=['MÃ CK','Name'], var_name='Năm')
df_sale_t_1 = df_sale_t_1.pivot(index=['MÃ CK','Năm'], columns='Name')

In [7]:
print(df_sale_t_1.columns)

MultiIndex([('value', '3. Doanh thu thuần (1)-(2)'),
            ('value',          'TỔNG CỘNG TÀI SẢN')],
           names=[None, 'Name'])


In [8]:
df_sale_t_1.columns = ['Doanh thu năm t-1', 'Tổng tài sản năm t-1']

In [9]:
df_sale_t_1

Doanh thu năm t-1  Tổng tài sản năm t-1
MÃ CK Năm                                          
AAA   2015       1.560644e+12          1.421651e+12
      2016       1.614549e+12          1.954765e+12
      2017       2.143770e+12          3.077616e+12
      2018       4.069608e+12          4.576157e+12
      2019       8.011573e+12          7.529167e+12
...                       ...                   ...
YEG   2016       4.040193e+11          3.496638e+11
      2017       5.626560e+11          3.545225e+11
      2018       8.403499e+11          6.573053e+11
      2019       1.676580e+12          1.961755e+12
      2020       1.452293e+12          1.515496e+12

[1938 rows x 2 columns]

In [10]:
array_info_DT = ['3. Doanh thu thuần (1)-(2)','7. Chi phí tài chính','-Trong đó: Chi phí lãi vay', '15. Tổng lợi nhuận kế toán trước thuế (11)+(14)', '19. Lợi nhuận sau thuế thu nhập doanh nghiệp (15)-(18)']
array_info_TS = ["A. Tài sản lưu động và đầu tư ngắn hạn", "III. Các khoản phải thu ngắn hạn","IV. Tổng hàng tồn kho","B. Tài sản cố định và đầu tư dài hạn",
                   "II. Tài sản cố định","3. Tài sản cố định vô hình","TỔNG CỘNG TÀI SẢN", "A. Nợ phải trả", "I. Nợ ngắn hạn","3. Phải trả người bán ngắn hạn",
                   "II. Nợ dài hạn"]
array_info_LCTT = ["Lưu chuyển tiền thuần từ hoạt động kinh doanh","8. Cổ tức, lợi nhuận đã trả cho chủ sở hữu",'- Khấu hao TSCĐ']
array_info = [array_info_DT, array_info_TS, array_info_LCTT]

In [11]:
url_DT = 'https://www.fireant.vn/api/Data/Finance/LastestFinancialReports?type=2&year=2021&quarter=0&count=6&symbol='
url_TS = 'https://svr5.fireant.vn/api/Data/Finance/LastestFinancialReports?type=1&year=2021&quarter=0&count=6&symbol='
url_LCTT = 'https://svr5.fireant.vn/api/Data/Finance/LastestFinancialReports?type=4&year=2021&quarter=0&count=6&symbol='
url_list = [url_DT, url_TS, url_LCTT]
df_sale = pd.DataFrame()
for index in range(0,len(list_cty)):
    df_financial = pd.DataFrame()
    for url_index in range(0, len(url_list)):
        url_temp = url_list[url_index] + list_cty[index]
        r = requests.get(url_temp)
        if r.status_code == 200:
            json_data = r.json()
        else:
            break
        if json_data != None:
            json_data = r.json()
            df = pd.DataFrame(json_data)
            df = df[df['Name'].isin(array_info[url_index])]
            df_financial = pd.concat([df_financial, df])
        else:
            df_financial = pd.DataFrame()
            break
    if df_financial.empty != True :
        ten_cty_list = []
        for number in range(0, len(df_financial['Name'])):
            ten_cty_list.append(list_cty[index])
        df_financial['MÃ CK'] = ten_cty_list
        df_sale = pd.concat([df_sale, df_financial])
        df_sale = df_sale[['Name','MÃ CK','Values']]
df_sale.reset_index(drop=True,inplace=True)
df_sale

,Name,MÃ CK,Values
0,3. Doanh thu thuần (1)-(2),AAA,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
1,7. Chi phí tài chính,AAA,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
2,-Trong đó: Chi phí lãi vay,AAA,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
3,15. Tổng lợi nhuận kế toán trước thuế (11)+(14),AAA,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
4,19. Lợi nhuận sau thuế thu nhập doanh nghiệp (...,AAA,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
...,...,...,...
6825,3. Phải trả người bán ngắn hạn,YEG,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
6826,II. Nợ dài hạn,YEG,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
6827,- Khấu hao TSCĐ,YEG,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."
6828,Lưu chuyển tiền thuần từ hoạt động kinh doanh,YEG,"[{'Period': '2015', 'Year': 2015, 'Quarter': 0..."


In [12]:
for index, row in df_sale.iterrows():
    years = [2015,2016,2017,2018,2019,2020]
    if(index in df_sale.index):
        frame = df_sale.loc[index,'Values']
        year_in_frame = []
        for i in range(0,len(frame)) :
            year_in_frame.append(frame[i]['Year'])
        if(years == year_in_frame):
            for i in range(0,len(frame)):
                df_sale.loc[index,frame[i]['Year']] = frame[i]['Value']
        else:
            df_sale = df_sale.drop(df_sale[df_sale['MÃ CK'] == df_sale.loc[index,'MÃ CK']].index)


In [13]:
df_sale = df_sale.drop(columns='Values')
df_sale = df_sale.melt(id_vars=['MÃ CK','Name'], var_name='Năm')
df_sale = df_sale.pivot(index=['MÃ CK','Năm'], columns='Name')

In [14]:
df_sale = df_sale.merge(df_sale_t_1,left_index=True, right_index=True)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [15]:
df_sale

(value, - Khấu hao TSCĐ)  (value, -Trong đó: Chi phí lãi vay)  \
MÃ CK Năm                                                                   
AAA   2015              8.335829e+10                         2.406581e+10   
      2016              9.295033e+10                         2.728447e+10   
      2017              1.680285e+11                         8.059205e+10   
      2018              2.449514e+11                         1.259805e+11   
      2019              2.586343e+11                         2.254216e+11   
...                              ...                                  ...   
YEG   2016              2.175104e+09                         1.278063e+10   
      2017              1.046891e+10                         1.618387e+10   
      2018              1.881233e+10                         1.210950e+10   
      2019              2.164515e+10                         1.729814e+10   
      2020              1.758253e+10                         3.532670e+09   

            (value, 15. Tổng lợi nhuận kế toán trước thuế (11)+(14))  \
MÃ CK Năm                                                              
AAA   2015                                       5.061492e+10          
      2016                                       1.665968e+11          
      2017                                       3.280543e+11          
      2018                                       2.539843e+11          
      2019                                       5.973934e+11          
...                                                       ...          
YEG   2016                                       2.210965e+10          
      2017                                       9.616478e+10          
      2018                                       2.153437e+11          
      2019                                      -3.864978e+11          
      2020                                      -1.677648e+11          

            (value, 19. Lợi nhuận sau thuế thu nhập doanh nghiệp (15)-(18))  \
MÃ CK Năm                                                                     
AAA   2015                                       4.054881e+10                 
      2016                                       1.429250e+11                 
      2017                                       2.633641e+11                 
      2018                                       2.121489e+11                 
      2019                                       4.910289e+11                 
...                                                       ...                 
YEG   2016                                       1.612673e+10                 
      2017                                       8.227913e+10                 
      2018                                       1.631038e+11                 
      2019                                      -3.828111e+11                 
      2020                                      -1.799989e+11                 

            (value, 3. Doanh thu thuần (1)-(2))  \
MÃ CK Năm                                         
AAA   2015                         1.614549e+12   
      2016                         2.143770e+12   
      2017                         4.069608e+12   
      2018                         8.011573e+12   
      2019                         9.258073e+12   
...                                         ...   
YEG   2016                         5.626560e+11   
      2017                         8.403499e+11   
      2018                         1.676580e+12   
      2019                         1.452293e+12   
      2020                         1.218619e+12   

            (value, 3. Phải trả người bán ngắn hạn)  \
MÃ CK Năm                                             
AAA   2015                             2.008857e+11   
      2016                             3.042635e+11   
      2017                             5.192982e+11   
      2018                             6.227787e+11   
      2019                             6.044452e+11

In [ ]:
url = 'https://e.cafef.vn/fi.ashx?symbol='
df_PE = pd.DataFrame()
for cty in list_cty :
    url_temp = url + cty
    r = requests.get(url_temp)
    if r.status_code == 200:
        json_data = r.json()
    else:
        break
    df = pd.DataFrame(json_data)
    if(df.empty != True and df[df['Year'].isin([2015,2016,2017,2018,2019,2020])]['Year'].count() == 6):
        df = df[df['Year'].isin([2015,2016,2017,2018,2019,2020])][['Year','Symbol','PE']]
        df_PE = pd.concat([df_PE, df])
print(df_PE)

In [ ]:
df_PE.columns = ['Năm','MÃ CK','P/E']
df_PE = df_PE.set_index(['MÃ CK','Năm'])

In [ ]:
df_sale = df_sale.merge(df_PE,left_index=True, right_index=True)

In [ ]:
df_sale

In [ ]:
df_sale.to_excel('Dataset_FI_2.xlsx')